In [75]:
!pip install numpy moabb

In [1]:
import numpy as np
from moabb.datasets import Cho2017
from moabb.paradigms import LeftRightImagery
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from mne.decoding import CSP
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import FilterBankLeftRightImagery, LeftRightImagery
from moabb.pipelines.utils import FilterBank

In [28]:
pipelines = {}
pipelines["CSP+LDA"] = make_pipeline(CSP(n_components=8), LDA())

In [29]:
# Load MOABB dataset with frequency band filtering
fmin = 8  # 최소 주파수 8Hz
fmax = 35  # 최대 주파수 35Hz
paradigm = LeftRightImagery(fmin=fmin, fmax=fmax, resample=128)


# Get data from the first subject and the first session
dataset = Cho2017()
X, y, metadata = paradigm.get_data(dataset=dataset, subjects=[1])

# Apply CSP to extract spatial features
csp = CSP(n_components=8)  # Extract 8 spatial components
X_csp = csp.fit_transform(X, y)  # Transform the data using CSP

Trials demeaned and stacked with zero buffer to create continuous data -- edge effects present
C:\Users\kimso\anaconda3\envs\csp_lda\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 200 events (all good), 0 – 3 s (baseline off), ~150.2 MiB, data loaded,
 'left_hand': 100
 'right_hand': 100>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\csp_lda\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 4.9e+03 (2.2e-16 eps * 64 dim * 3.5e+17  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=left_hand covariance using EMPIRICAL
Done.
Estimating class=right_hand covariance using EMPIRICAL
Done.


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_csp, y, test_size=0.3, random_state=42)

# Train logistic regression model
clf_lr = LogisticRegression()
clf_lr.fit(X_train, y_train)

# Make predictions and evaluate
predict_lr = clf_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, predict_lr)
print(f"Logistic Regression 정확도 (Accuracy): {accuracy_lr:.2f}")

Logistic Regression 정확도 (Accuracy): 0.87


In [31]:
clf_lda = LDA()
clf_lda.fit(X_train, y_train)

# Make predictions and evaluate
predict_lda = clf_lda.predict(X_test)
accuracy_lda = accuracy_score(y_test, predict_lda)
print(f"LDA 정확도 (Accuracy): {accuracy_lda:.2f}")

LDA 정확도 (Accuracy): 0.88
